# Clustering Crypto

In [650]:
# Initial imports
import requests
import pandas as pd
import matplotlib.pyplot as plt
import hvplot.pandas
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans



from pathlib import Path

### Fetching Cryptocurrency Data

In [651]:
# Alternatively, use the provided csv file:
file_path = Path("crypto_data.csv")
# Create a DataFrame
# Keep only necessary columns:
# 'CoinName','Algorithm','IsTrading','ProofType','TotalCoinsMined','TotalCoinSupply'
crypto_df = pd.read_csv(file_path,index_col="Unnamed: 0")

### Data Preprocessing

In [652]:
# Keep only cryptocurrencies that are trading
crypto_df=crypto_df[crypto_df["IsTrading"]==True]
crypto_df.head()


,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [653]:
# Keep only cryptocurrencies with a working algorithm
# Remove rows with at least 1 null value
# Drop rows where there are 'N/A' text values
crypto_df=crypto_df.dropna()
crypto_df

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,True,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,True,PoW,1.792718e+07,21000000
...,...,...,...,...,...,...
ZEPH,ZEPHYR,SHA-256,True,DPoS,2.000000e+09,2000000000
GAP,Gapcoin,Scrypt,True,PoW/PoS,1.493105e+07,250000000
BDX,Beldex,CryptoNight,True,PoW,9.802226e+08,1400222610
ZEN,Horizen,Equihash,True,PoW,7.296538e+06,21000000


In [654]:
# Remove the 'IsTrading' column
crypto_df=crypto_df.drop(columns="IsTrading")
crypto_df.head()


,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
808,808,SHA-256,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000


In [655]:
# Remove rows with cryptocurrencies having no coins mined
crypto_df=crypto_df[crypto_df["TotalCoinsMined"] != 0]
crypto_df.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,PoW,1.076842e+08,0


In [656]:
# Store the 'CoinName'column in its own DataFrame prior to dropping it from crypto_df 
coins_name=crypto_df.index

In [657]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm
crypto_df=crypto_df.drop(columns="CoinName")

In [658]:

#le = LabelEncoder()
#le.fit(crypto_df["Algorithm"])
#crypto_df["Algorithm"] = le.transform(crypto_df["Algorithm"])

#le.fit(crypto_df["ProofType"])
#crypto_df["ProofType"] = le.transform(crypto_df["ProofType"])

#crypto_df

In [659]:
# Create dummy variables for text features
X=pd.get_dummies(crypto_df,columns=["Algorithm","ProofType"])
X

,TotalCoinsMined,TotalCoinSupply,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,Algorithm_C11,...,ProofType_PoW/PoS,ProofType_PoW/PoS,ProofType_PoW/PoW,ProofType_PoW/nPoS,ProofType_Pos,ProofType_Proof of Authority,ProofType_Proof of Trust,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW
42,4.199995e+01,42,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
404,1.055185e+09,532000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1337,2.927942e+10,314159265359,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BTC,1.792718e+07,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ETH,1.076842e+08,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZEPH,2.000000e+09,2000000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
GAP,1.493105e+07,250000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BDX,9.802226e+08,1400222610,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ZEN,7.296538e+06,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [660]:
x_scaled= StandardScaler().fit_transform(X)
print(X[0:5])

      TotalCoinsMined TotalCoinSupply  Algorithm_1GB AES Pattern Search  \
42       4.199995e+01              42                                 0   
404      1.055185e+09       532000000                                 0   
1337     2.927942e+10    314159265359                                 0   
BTC      1.792718e+07        21000000                                 0   
ETH      1.076842e+08               0                                 0   

      Algorithm_536  Algorithm_Argon2d  Algorithm_BLAKE256  Algorithm_Blake  \
42                0                  0                   0                0   
404               0                  0                   0                0   
1337              0                  0                   0                0   
BTC               0                  0                   0                0   
ETH               0                  0                   0                0   

      Algorithm_Blake2S  Algorithm_Blake2b  Algorithm_C11  ...  \
42      

### Reducing Dimensions Using PCA

In [661]:
# Use PCA to reduce dimensions to 3 principal components
pca = PCA(n_components=3)

In [662]:
# Create a DataFrame with the principal components data
x_pca = pca.fit_transform(x_scaled)

pcs_df= pd.DataFrame(
    data=x_pca, columns=["PC1", "PC2","PC3"]
)
pcs_df=pcs_df.set_index(coins_name)
pcs_df

,PC1,PC2,PC3
42,-0.333262,1.016306,-0.512337
404,-0.316596,1.016281,-0.512724
1337,2.309608,1.564403,-0.569810
BTC,-0.144397,-1.317876,0.217527
ETH,-0.140816,-1.960368,0.315132
...,...,...,...
ZEPH,2.480652,0.878306,0.103639
GAP,-0.331306,1.016170,-0.512352
BDX,0.330082,-2.286269,0.386753
ZEN,-0.148433,-2.110599,0.318861


### Clustering Crytocurrencies Using K-Means

#### Finding the Best Value for `k` Using the Elbow Curve

In [663]:
inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range of k values
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(pcs_df)
    inertia.append(km.inertia_)

# Create the Elbow Curve using hvPlot
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", xticks=k, title="Elbow Curve")



C:\Users\silver_empire\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:1036: UserWarning:

KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=3.



:Curve   [k]   (inertia)

Running K-Means with `k=<your best value for k here>`

In [664]:
# Initialize the K-Means model

# Fit the model

# Predict clusters

# Create a new DataFrame including predicted clusters and cryptocurrencies features
def get_clusters(k, data):
    # Initialize the K-Means model
    model = KMeans(n_clusters=k, random_state=0)

    # Fit the model
    model.fit(data)

    # Predict clusters
    predictions = model.predict(data)

    # Create return DataFrame with predicted clusters
    data["class"] = model.labels_

    return data


In [665]:
clustered_df1=get_clusters(5,pcs_df)
clustered_df=crypto_df.join(clustered_df1)
clustered_df["CoinName"]=coins_name
clustered_df=clustered_df[["Algorithm","ProofType","TotalCoinsMined","TotalCoinSupply","PC1","PC2","PC3","CoinName","class"]]
clustered_df.head(10)

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC1,PC2,PC3,CoinName,class
42,Scrypt,PoW/PoS,4.199995e+01,42,-0.333262,1.016306,-0.512337,42,1
404,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.316596,1.016281,-0.512724,404,1
1337,X13,PoW/PoS,2.927942e+10,314159265359,2.309608,1.564403,-0.569810,1337,4
BTC,SHA-256,PoW,1.792718e+07,21000000,-0.144397,-1.317876,0.217527,BTC,0
ETH,Ethash,PoW,1.076842e+08,0,-0.140816,-1.960368,0.315132,ETH,0
LTC,Scrypt,PoW,6.303924e+07,84000000,-0.163376,-1.093100,-0.024486,LTC,0
DASH,X11,PoW/PoS,9.031294e+06,22000000,-0.397732,1.159734,-0.393624,DASH,1
XMR,CryptoNight-V7,PoW,1.720114e+07,0,-0.151895,-2.243973,0.383584,XMR,0
ETC,Ethash,PoW,1.133597e+08,210000000,-0.139257,-1.960484,0.315122,ETC,0
ZEC,Equihash,PoW,7.383056e+06,21000000,-0.148432,-2.110599,0.318861,ZEC,0


### Visualizing Results

#### 3D-Scatter with Clusters

In [666]:
# Create a 3D-Scatter with the PCA data and the clusters
fig = px.scatter_3d(clustered_df, x='PC1', y='PC2', z='PC3',color="ProofType",hover_name="CoinName",hover_data=["Algorithm"])
fig.show()

#### Table of Tradable Cryptocurrencies

In [677]:
# Table with tradable cryptos
tradeable_df=clustered_df.hvplot.table(columns=["CoinName", "Algorithm", "ProofType", "TotalCoinSupply", "TotalCoinsMined", "class"])
tradeable_df

:Table   [CoinName,Algorithm,ProofType,TotalCoinSupply,TotalCoinsMined,class]

In [678]:
# Print the total number of tradable cryptocurrencies
tradeable_df.shape[0]

533

#### Scatter Plot with Tradable Cryptocurrencies

In [680]:
# Scale data to create the scatter plot
# Plot the scatter with x="TotalCoinsMined" and y="TotalCoinSupply"
fig = px.scatter(clustered_df, x='TotalCoinsMined', y='TotalCoinSupply',color="ProofType",hover_name="CoinName",hover_data=["Algorithm"])
fig.show()
